In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') # For plots

In [2]:
df = pd.read_csv(r"C:\Users\V0V01S7\Downloads\robert.csv")

In [3]:
df['ACTV_START_TS'] = pd.to_datetime(df['ACTV_START_TS'])
df['ACTL_ACTV_END_TS'] = pd.to_datetime(df['ACTL_ACTV_END_TS'])
df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
df['dayhour_end'] = df["ACTL_ACTV_END_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
time_line = pd.date_range(df["dayhour"].min(),df["dayhour"].max(),freq='H')
time_line = time_line.to_series()
ts = pd.DataFrame(time_line,columns = ["dayhour"])
ts = ts.set_index("dayhour")
ts = pd.DataFrame(time_line,columns = ["dayhour"])
ts = ts.set_index("dayhour")
ts = pd.merge(ts,df,how="left",on=["dayhour"])
ts["ACTV_DUR_MIN_QTY"] = ts["ACTV_DUR_MIN_QTY"].fillna(0)

In [4]:
ts.head()

,dayhour,ACTV_START_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,dayhour_end
0,2021-07-27 04:00:00,2021-07-27 04:31:10.007,2021-07-27 04:33:41.730,2.52,2021-07-27 04:00:00
1,2021-07-27 04:00:00,2021-07-27 04:33:41.730,NaT,0.00,NaT
2,2021-07-27 05:00:00,NaT,NaT,0.00,NaT
3,2021-07-27 06:00:00,2021-07-27 06:16:14.438,2021-07-27 07:15:36.744,59.37,2021-07-27 07:00:00
4,2021-07-27 07:00:00,2021-07-27 07:15:36.744,NaT,0.00,NaT


In [6]:
ts = ts.drop(columns=["ACTL_ACTV_END_TS"])
ts.head()

,dayhour,ACTV_START_TS,ACTV_DUR_MIN_QTY,dayhour_end
0,2021-07-27 04:00:00,2021-07-27 04:31:10.007,2.52,2021-07-27 04:00:00
1,2021-07-27 04:00:00,2021-07-27 04:33:41.730,0.00,NaT
2,2021-07-27 05:00:00,NaT,0.00,NaT
3,2021-07-27 06:00:00,2021-07-27 06:16:14.438,59.37,2021-07-27 07:00:00
4,2021-07-27 07:00:00,2021-07-27 07:15:36.744,0.00,NaT


In [7]:
for i in range(len(ts)-1):
        if ts["ACTV_START_TS"][i].replace(minute=0, second=0, microsecond=0) == ts["dayhour"][i+1] and ts["dayhour"][i+1] == ts["dayhour_end"][i+1]:
            ts["ACTV_DUR_MIN_QTY"][i+1] = ts["ACTV_DUR_MIN_QTY"][i] + ts["ACTV_DUR_MIN_QTY"][i+1]
            continue
        if ts["ACTV_DUR_MIN_QTY"][i] > (ts["dayhour"][i+1]-ts["ACTV_START_TS"][i]).total_seconds()/60:
            temp = ((ts["dayhour"][i+1]-ts["ACTV_START_TS"][i]).total_seconds())/60
            ts["ACTV_DUR_MIN_QTY"][i+1] = ts["ACTV_DUR_MIN_QTY"][i]-temp
            ts["ACTV_DUR_MIN_QTY"][i] = temp
            ts["ACTV_START_TS"][i+1] = ts["dayhour"][i+1]
            
ts = ts.drop(columns="ACTV_START_TS")
ts = ts.set_index("dayhour")
ts = ts.drop(ts.tail(1).index)
ts.head()

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_16456\3835171086.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts["ACTV_DUR_MIN_QTY"][i+1] = ts["ACTV_DUR_MIN_QTY"][i]-temp
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_16456\3835171086.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts["ACTV_DUR_MIN_QTY"][i] = temp
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_16456\3835171086.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

,ACTV_DUR_MIN_QTY,dayhour_end
dayhour,,
2021-07-27 04:00:00,-31.166783,2021-07-27 04:00:00
2021-07-27 04:00:00,33.686783,NaT
2021-07-27 05:00:00,0.000000,NaT
2021-07-27 06:00:00,43.759367,2021-07-27 07:00:00
2021-07-27 07:00:00,15.610633,NaT


In [8]:
fs = ts.groupby("dayhour").max()
fs

,ACTV_DUR_MIN_QTY,dayhour_end
dayhour,,
2021-07-27 04:00:00,33.686783,2021-07-27 04:00:00
2021-07-27 05:00:00,0.000000,NaT
2021-07-27 06:00:00,43.759367,2021-07-27 07:00:00
2021-07-27 07:00:00,15.610633,NaT
2021-07-27 08:00:00,0.000000,NaT
...,...,...
2022-07-01 06:00:00,45.916533,2022-07-01 07:00:00
2022-07-01 07:00:00,7.063467,2022-07-01 07:00:00
2022-07-01 08:00:00,50.342067,2022-07-01 08:00:00


In [12]:
#fs = fs.drop(columns=["dayhour_end"])


In [11]:
fs.loc[fs["ACTV_DUR_MIN_QTY"]>60]

,ACTV_DUR_MIN_QTY
dayhour,
2021-07-28 06:00:00,63.45
2021-08-04 07:00:00,64.98
2021-08-07 12:00:00,104.64
2021-08-10 11:00:00,111.15
2021-08-12 07:00:00,103.25
...,...
2022-03-20 11:00:00,91.45
2022-03-21 09:00:00,86.25
2022-03-25 05:00:00,60.04


In [5]:
features_and_target.head()

NameError: name 'features_and_target' is not defined